<a href="https://colab.research.google.com/github/leti-olabarri/premier-fantasy/blob/feature%2Fload_data/models/Classification_with_more_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing

import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np

from sklearn.model_selection import train_test_split

In [2]:
!wget /resources/data/all.txt https://drive.google.com/file/d/1IH1tCQRbJBJGTnUxKNnCwvhLI9IPVCnf/view?usp=sharing

/resources/data/all.txt: Scheme missing.
--2021-10-24 09:04:30--  https://drive.google.com/file/d/1IH1tCQRbJBJGTnUxKNnCwvhLI9IPVCnf/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 142.250.141.100, 142.250.141.138, 142.250.141.101, ...
Connecting to drive.google.com (drive.google.com)|142.250.141.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘view?usp=sharing’

view?usp=sharing        [ <=>                ]  65.16K  --.-KB/s    in 0.03s   

2021-10-24 09:04:31 (2.33 MB/s) - ‘view?usp=sharing’ saved [66726]

FINISHED --2021-10-24 09:04:31--
Total wall clock time: 0.5s
Downloaded: 1 files, 65K in 0.03s (2.33 MB/s)


In [7]:
def starts_with_date_and_time(string):
    pattern = '^([0-9]+)(/)([0-9]+)(/)([0-9][0-9]) ([0-9]+):([0-9][0-9]) -'
    result = re.match(pattern, string)
    if result:
        return True
    return False

def get_data_point(line):   
    splitLine = line.split(' - ')
    dateTime = splitLine[0]
    date, time = dateTime.split(' ') 
    message = ' '.join(splitLine[1:])
    splitMessage = message.split(': ')
    author = splitMessage[0]
    message = ' '.join(splitMessage[1:])
    return date, time, author, message

parsedData = [] # List to keep track of data so it can be used by a Pandas dataframe
### Uploading exported chat file
conversationPath = '../content/drive/MyDrive/data/all.txt' # chat file
with open(conversationPath, encoding="utf-8") as fp:
    ### Skipping first line of the file because contains information related to something about end-to-end encryption
    fp.readline() 
    messageBuffer = [] 
    date, time, author = None, None, None
    while True:
        line = fp.readline() 
        if not line: 
            break
        line = line.strip()
        if starts_with_date_and_time(line):
            if len(messageBuffer) > 0: 
                parsedData.append([date, time, author, ' '.join(messageBuffer)]) 
            messageBuffer.clear()
            date, time, author, message = get_data_point(line) 
            messageBuffer.append(message)
        else:
            messageBuffer.append(line)
df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message']) # Initialising a pandas Dataframe.
### changing datatype of "Date" column.
df["Date"] = pd.to_datetime(df["Date"])
df["Time"] = pd.to_datetime(df["Time"], format='%H%M', errors='ignore')

In [8]:
df.head()

,Date,Time,Author,Message
0,2019-12-20,21:17,Leticia Olábarri,Adri
1,2019-12-20,21:17,Leticia Olábarri,¿Vienes?
2,2019-12-20,22:02,Adri Ávalos,Sí 😬
3,2019-12-20,22:03,Adri Ávalos,Ya voy
4,2019-12-20,22:03,Adri Ávalos,Me he liado un pelin


In [9]:
df.drop(df[(df['Author'] == 'Los mensajes y las llamadas están cifrados de extremo a extremo. Nadie fuera de este chat, ni siquiera WhatsApp, puede leerlos ni escucharlos. Toca para obtener más información.')].index, inplace=True)
df.drop(df[(df['Author'] == '\u200eMarcela Duque cambió su número de teléfono. Toca para enviar un mensaje o añadir el nuevo número.')].index, inplace=True)
df.drop(df[(df['Author'] == '\u200eMarcela Duque cambió su número de teléfono. Actualmente te estás comunicando con su nuevo número. Toca para añadir a tus contactos.')].index, inplace=True)

In [10]:
df.drop(df[(df['Message'] == '<Multimedia omitido>')].index, inplace=True)

In [11]:
df["Message total length"] = df["Message"].apply(lambda x: len(x))
df.drop(df[(df['Message total length'] < 10)].index, inplace=True)

In [12]:
def count_words(string):
    word_list = string.split()

    number_of_words = len(word_list)
    return number_of_words

df["Number of words"] = df["Message"].apply(lambda x: count_words(x))

In [13]:
def count_intro_signs(string):
    count = 0
    for i in string:
        if (i == "¿") or (i == "¡"):
            count = count + 1
    return count

df["Number of intro signs (¿ or ¡)"] = df["Message"].apply(lambda x: count_intro_signs(x))

In [14]:
def count_stops_and_commas(string):
    count = 0
    for i in string:
        if (i == ".") or (i == ","):
            count = count + 1
    return count

df["Number of stops and commas"] = df["Message"].apply(lambda x: count_stops_and_commas(x))

In [20]:
def count_strange_words(string):
    count = 0
    word_list = string.split()
    if (" xfa " in word_list) or (" q " in word_list):
        count = count + 1
    return count

df["Number of attypical abbreviations"] = df["Message"].apply(lambda x: count_strange_words(x))

In [21]:
df.head()

,Date,Time,Author,Message,Message total length,Number of words,Number of intro signs (¿ or ¡),Number of stops and commas,Number of attypical abbreviations
4,2019-12-20,22:03,0,Me he liado un pelin,20,5,0,0,0
5,2019-12-20,22:09,1,Vaaaaaamooooooossssss,21,1,0,0,0
7,2019-12-20,22:14,1,Avisa cuando llegues,20,3,0,0,0
9,2019-12-20,22:15,0,Por dónde estáis ?,18,4,0,0,0
11,2019-12-20,22:15,1,Enfrente del escenario,22,3,0,0,0


In [22]:
df.shape

(43096, 9)

In [23]:
def transform_names(name):
    if name == "Leticia Olábarri":
        return 1
    else:
        return 0

df["Author"] = df["Author"].apply(transform_names)

In [24]:
df.head()

,Date,Time,Author,Message,Message total length,Number of words,Number of intro signs (¿ or ¡),Number of stops and commas,Number of attypical abbreviations
4,2019-12-20,22:03,0,Me he liado un pelin,20,5,0,0,0
5,2019-12-20,22:09,0,Vaaaaaamooooooossssss,21,1,0,0,0
7,2019-12-20,22:14,0,Avisa cuando llegues,20,3,0,0,0
9,2019-12-20,22:15,0,Por dónde estáis ?,18,4,0,0,0
11,2019-12-20,22:15,0,Enfrente del escenario,22,3,0,0,0


In [25]:
X = df.drop(columns=["Author", "Date", "Time"])

In [26]:
X.head()

,Message,Message total length,Number of words,Number of intro signs (¿ or ¡),Number of stops and commas,Number of attypical abbreviations
4,Me he liado un pelin,20,5,0,0,0
5,Vaaaaaamooooooossssss,21,1,0,0,0
7,Avisa cuando llegues,20,3,0,0,0
9,Por dónde estáis ?,18,4,0,0,0
11,Enfrente del escenario,22,3,0,0,0


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, df["Author"], test_size=0.2, random_state=42)

In [28]:
X_train.shape

(34476, 6)

In [29]:
X_test.shape

(8620, 6)

In [30]:
y_train.shape

(34476,)

In [31]:
y_test.shape

(8620,)

In [32]:
y_test.head()

13330    0
23560    0
22477    0
45727    0
52977    0
Name: Author, dtype: int64

In [33]:
labels = y_train.unique()
labels

array([0])

In [34]:
X_train_feat = X_train.drop(columns=["Message"])
X_test_feat = X_test.drop(columns=["Message"])
X_train_mess = X_train["Message"]
X_test_mess = X_test["Message"]

In [35]:
max_features = 10000
sequence_length = 1000

vectorize_layer = layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [36]:
vectorize_layer.adapt(X_train_mess)

In [37]:
train_ds_mess = vectorize_layer(X_train_mess)
test_ds_mess = vectorize_layer(X_test_mess)

In [38]:
embedding_dim = 16

model_vector = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

model_vector.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160016    
_________________________________________________________________
dropout (Dropout)            (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 160,033
Trainable params: 160,033
Non-trainable params: 0
_________________________________________________________________
